In [1]:
from astropy.coordinates import SkyCoord, get_moon, EarthLocation, ICRS, GCRS,AltAz
from astropy.io.fits.hdu.hdulist import HDUList
from datetime import date
from astral import Astral, Location
from astropy.time import Time
import astropy.units as u

In [12]:
import pyfits
import pandas as pd
import numpy as np
from itertools import chain
from collections import Counter
from scipy.interpolate import interp1d
from datetime import datetime
from astropy.time import Time

In [13]:
## Finding out which file years to download
all_sky=pd.read_csv('objSKY.csv')
all_sky.head()
print('earliest date=', min(all_sky['MJD']), 'or 12-11-2009')
print('latest date=', max(all_sky['MJD']), 'or 05-12-2016')

earliest date= 55176 or 12-11-2009
latest date= 57520 or 05-12-2016


## Data

In [14]:
## To check for reading in the right file
rand_files=('spec-3586-55181-0496.fits','spec-3586-55181-0788.fits','spec-3586-55181-0996.fits',
            'spec-3761-55272-0008.fits','spec-10000-57346-0334.fits','spec-3761-55272-0475.fits',
            'spec-10000-57346-0659.fits','spec-5478-56014-0654.fits','spec-10000-57346-0955.fits',
            'spec-5478-56014-0716.fits','spec-10000-57346-0994.fits')

## To look for patterns
inseq_files=('spec-3663-55176-0010.fits','spec-3663-55176-0012.fits','spec-3663-55176-0020.fits',
             'spec-3663-55176-0024.fits','spec-3663-55176-0036.fits','spec-3663-55176-0038.fits',
             'spec-3663-55176-0048.fits','spec-3663-55176-0052.fits','spec-3663-55176-0056.fits',
             'spec-3663-55176-0068.fits','spec-3663-55176-0075.fits','spec-3663-55176-0078.fits',
             'spec-3663-55176-0090.fits','spec-3663-55176-0094.fits','spec-3663-55176-0096.fits',
             'spec-3663-55176-0108.fits','spec-3663-55176-0112.fits','spec-3663-55176-0114.fits',
             'spec-3663-55176-0128.fits','spec-3663-55176-0134.fits')

solar_files=('g2009.txt','g2010.txt','g2011.txt','g2012.txt','g2013.txt','g2014.txt','g2015.txt','g2016.txt')

## Getting MJDs from fits

In [15]:
#a) first load all files and convert all dates into mjds.
def get_mjd(file):
    da=pyfits.open(file)
    return (da[4].header['MJD'])

In [16]:
for i in rand_files:
    print(get_mjd(i))

55181
55181
55181
55272
57345
55272
57345
56014
57345
56014
57345


## Master List of Solar File MJDs & Coverage

In [38]:
def merge_subs(lst_of_lsts):
    res = []
    for row in lst_of_lsts:
        for i, resrow in enumerate(res):
            if row[:3]==resrow[:3]:
                res[i] += row[1:]
                break
        else:
            res.append(row)
    return (res)
## Merges lists, but does not remove first 3 cols

In [39]:
def f2(seq): 
    # order preserving
    checked = []
    for e in seq:
        if e not in checked:
            checked.append(e)
    return checked
## removes duplicates

In [53]:
## Function to organize mill. of sol. disk by date
def mjds_sol(txt_file):
    ## We only need year, month, day of month, and mill. of sol. disk
    ## Cols         1-4    5-6      7-8               31-34
    demo=list(chain.from_iterable((x[:4], x[4:6], x[6:10], x[31:34]) for x in open(txt_file).readlines()))
    ## Organizing cols
    str_list=[demo[i*4:i*4+4] for i in range(int(len(demo)/4))]
    ## Converting to float
    float_dates=[]
    for k in range(len(str_list)):
        float_dates.append([float(i) for i in str_list[k]])
    b=([x[:4] for x in float_dates])
    ve=merge_subs(b)
    new=[]
    summed_mills=[]
    no_mills=[]
    merged_lists=[]
    merged_utc=[]
    for i in range(len(ve)):
        new.append(f2(ve[i]))
        summed_mills.append(sum(new[i][3:]))
        no_mills.append(new[i][:3])
        merged_lists.append(np.append(no_mills[i],summed_mills[i]).tolist())
        ## Conversion to datetime ##
        merged_lists[i][3:3]=[12]
        merged_utc.append([int(n) for n in merged_lists[i]])
    utc_df=pd.DataFrame(merged_utc)
    utc_df.columns=['year','month','day','hour','mill']
    no_mills_mjds=[]
    vt=[]
    merged_mjds=[]
    ## Conversion to MJDs ##
    for i in range(len(utc_df)):
        vt.append(Time(datetime(utc_df['year'][i],utc_df['month'][i],utc_df['day'][i],utc_df['hour'][i]), scale='utc'))
        no_mills_mjds.append(vt[i].mjd)
        merged_mjds.append(np.append(no_mills_mjds[i],summed_mills[i]).tolist())
    return(merged_mjds)

In [57]:
%%time
master_lst=[]
for i in solar_files:
    master_lst.append(mjds_sol(i))

CPU times: user 2.11 s, sys: 4 ms, total: 2.12 s
Wall time: 2.11 s


In [59]:
## flatten list
master_list=[item for sublist in master_lst for item in sublist]

In [65]:
len(master_list)

2498

## Interpolated function from MJDs to mill of sol. disk

In [66]:
master_df=pd.DataFrame(master_list)
master_df.columns=['mjd','sun_cov']

In [67]:
master_df[:8]

,mjd,sun_cov
0,54840.5,30.0
1,54841.5,51.0
2,54842.5,92.0
3,54843.5,18.0
4,54844.5,17.0
5,54850.5,18.0
6,54873.5,9.0
7,54874.5,13.0


In [72]:
mjd=master_df['mjd']
sun_cov=master_df['sun_cov']
#y=merged_lists[5]
ifunc=interp1d(mjd,sun_cov)
ifunc(54849)

array(17.75)

In [90]:
mjd[0]+1

54841.5

In [92]:
for i in range(len(master_df)):
    if mjd[i] != mjd[i+1]:#filling in missing noons
        print(mjd[i])
#        print(master_df['mjd'][i-1], master_df['mjd'][i])
#        print(interp1d(merged_lists[i-1], merged_lists[i]))

54840.5
54841.5
54842.5
54843.5
54844.5
54850.5
54873.5
54874.5
54875.5
54886.5
54887.5
54888.5
54896.5
54897.5
54942.5
54950.5
54951.5
54964.5
54965.5
54966.5
54967.5
54968.5
54969.5
54970.5
54974.5
54982.5
54983.5
54984.5
54985.5
54986.5
54987.5
54990.5
54991.5
54994.5
54999.5
55003.5
55004.5
55005.5
55006.5
55015.5
55016.5
55017.5
55018.5
55019.5
55020.5
55021.5
55022.5
55074.5
55075.5
55095.5
55096.5
55097.5
55098.5
55099.5
55100.5
55101.5
55102.5
55103.5
55104.5
55105.5
55124.5
55127.5
55128.5
55129.5
55130.5
55131.5
55132.5
55133.5
55134.5
55140.5
55141.5
55142.5
55144.5
55145.5
55146.5
55147.5
55150.5
55151.5
55153.5
55154.5
55155.5
55156.5
55157.5
55174.5
55175.5
55176.5
55177.5
55178.5
55179.5
55180.5
55181.5
55182.5
55183.5
55184.5
55185.5
55186.5
55187.5
55188.5
55189.5
55191.5
55192.5
55193.5
55194.5
55195.5
55196.5
55197.5
55198.5
55199.5
55200.5
55201.5
55203.5
55204.5
55205.5
55206.5
55207.5
55208.5
55209.5
55210.5
55211.5
55212.5
55213.5
55214.5
55216.5
55217.5
55218.5


KeyError: 2498

## Trial codes

In [322]:
#b) Then set up one massive interpolated function from mjds to mill of sol. disk
## Pulling out cols of interest
demo=list(chain.from_iterable((x[:4], x[4:6], x[6:10], x[31:34]) for x in open('g2010.txt').readlines()))
## Organizing cols
str_list=[demo[i*4:i*4+4] for i in range(int(len(demo)/4))]
## Converting to float
float_dates=[]
for k in range(len(str_list)):
    float_dates.append([float(i) for i in str_list[k]])
b=([x[:4] for x in float_dates])
ve=merge_subs(b)
new=[]
summed_mills=[]
no_mills=[]
merged_lists=[]
merged_utc=[]
for i in range(len(ve)):
    new.append(f2(ve[i]))
    summed_mills.append(sum(new[i][3:]))
    no_mills.append(new[i][:3])
    merged_lists.append(np.append(no_mills[i],summed_mills[i]).tolist())
    ## Conversion to datetime ##
    merged_lists[i][3:3]=[12]
    merged_utc.append([int(n) for n in merged_lists[i]])
utc_df=pd.DataFrame(merged_utc)
utc_df.columns=['year','month','day','hour','mill']
no_mills_mjds=[]
vt=[]
merged_mjds=[]
## Conversion to MJDs ##
for i in range(len(utc_df)):
    vt.append(Time(datetime(utc_df['year'][i],utc_df['month'][i],utc_df['day'][i],utc_df['hour'][i]), scale='utc'))
    no_mills_mjds.append(vt[i].mjd)
    merged_mjds.append(np.append(no_mills_mjds[i],summed_mills[i]).tolist())

In [329]:
#list_example=[[1,1,2016,212], [1,2,2016,170], [1,3,2016,150], [1,5,2016,96], [1,6,2016,150], [1,8,2016,321]]

In [330]:
#zip(list_example)